In [1]:
import ml_collections
import copy
import numpy as np
import matplotlib.pyplot as plt
import yaml
from tqdm import tqdm
import os
import torch
import torch.nn as nn
from os import path as pt
import pickle
from torch.utils.data import DataLoader, TensorDataset
from src.evaluation.summary import full_evaluation
from src.utils import set_seed, save_obj, load_obj

In [2]:
with open("./data/ref_log_return.pkl", "rb") as f:
    loaded_array = pickle.load(f)
train_log_return = torch.tensor(loaded_array)
print(train_log_return.shape)

with open("./data/ref_price.pkl", "rb") as f:
    loaded_array = pickle.load(f)
train_init_price = torch.tensor(loaded_array)
print(train_init_price.shape)

torch.Size([8937, 24, 3])
torch.Size([8937, 1, 3])


### Generative models for time series generation

In [3]:
# Load configuration dict
config_dir = 'configs/config.yaml'
with open(config_dir, encoding='utf-8') as file:
    config = ml_collections.ConfigDict(yaml.safe_load(file))
    
set_seed(config.seed)

if (config.device ==
        "cuda" and torch.cuda.is_available()):
    config.update({"device": "cuda:0"}, allow_val_change=True)
else:
    config.update({"device": "cpu"}, allow_val_change=True)
    
class XYDataset(TensorDataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        self.shape = X.shape

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.Y[index]

### Data Construction

We divide the data into training and validation set for the offline evaluation of our model

In [4]:
perm_idx = torch.randperm(train_log_return.shape[0])
train_size = int(0.8*train_log_return.shape[0])

cv_training_data = train_log_return[perm_idx[:train_size]].to(config.device).to(torch.float)
cv_init_price = train_init_price[perm_idx[:train_size]].to(config.device).to(torch.float)
cv_validation_data = train_log_return[perm_idx[train_size:]].to(config.device).to(torch.float)
cv_val_init_price = train_init_price[perm_idx[train_size:]].to(config.device).to(torch.float)

print(cv_training_data.shape, cv_init_price.shape, cv_validation_data.shape, cv_val_init_price.shape)

torch.Size([7149, 24, 3]) torch.Size([7149, 1, 3]) torch.Size([1788, 24, 3]) torch.Size([1788, 1, 3])


In [5]:
# Load the dataset
training_set = TensorDataset(cv_init_price, cv_training_data)

print(training_set[0][0].shape, training_set[0][1].shape)

train_dl = DataLoader(
    training_set,
    batch_size=config.batch_size,
    shuffle=True
)

for batch in train_dl:
    print(batch[0].shape, batch[1].shape)
    break
len(train_dl)

torch.Size([1, 3]) torch.Size([24, 3])
torch.Size([256, 1, 3]) torch.Size([256, 24, 3])


28

### Generative model

Here we construct a generator and a discriminator for this task. Both the generator and discriminator takes as input the time series. Then we have the training algorithm TailGANTrainer.

In [6]:
from src.baselines.Diffusion import * 

Done


In [7]:
input_dim =  3 # 24 hours x 3 cryptocurrencies
hidden_dim = 512
timesteps = 100
batch_size = config.batch_size
epochs = 5
set_seed(42)
betas = linear_beta_schedule(timesteps)
model = ScoreNetwork(input_dim=input_dim, hidden_dim=hidden_dim).to(device)
trainer = DDPM(model, betas=betas, train_dl=train_dl, timesteps=timesteps)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [8]:
trainer.train(optimizer=optimizer, epochs=epochs)
save_obj(trainer.model.state_dict(), './submission_bundle/model_dict.pkl')

Epoch 1/5, Average Loss: 215.4932
Epoch 2/5, Average Loss: 0.2517
Epoch 3/5, Average Loss: 0.1616
Epoch 4/5, Average Loss: 0.1468
Epoch 5/5, Average Loss: 0.1365


0

### Synthetic data generation

In [13]:
state_dict = load_obj('./submission_bundle/model_dict.pkl')

trainer.model.load_state_dict(state_dict)

trainer.model.eval()

eval_size = 1800

with torch.no_grad():
     generated_sample = trainer.sample(eval_size)

print(generated_sample.shape)
print(generated_sample)
save_obj(generated_sample, './submission_bundle/fake_log_return.pkl')

torch.Size([1800, 24, 3])
tensor([[[-1.6483e-03,  1.0965e-03,  9.9535e-04],
         [-1.2715e-03,  1.0834e-03,  5.0533e-04],
         [-1.3431e-03,  9.8036e-04,  5.3993e-04],
         ...,
         [ 6.3318e-03, -1.0101e-02,  8.0456e-03],
         [-1.2682e-03,  1.1004e-03,  5.9748e-04],
         [-2.4880e-03,  7.7853e-04, -5.7324e-04]],

        [[-7.0275e-04,  1.3481e-03,  6.5233e-04],
         [-1.2286e-03,  1.1093e-03,  4.7122e-04],
         [-1.1500e-03,  1.0593e-03,  6.0745e-04],
         ...,
         [-9.3809e-04,  8.4386e-04,  7.6436e-04],
         [-5.2958e-03,  7.2353e-03, -1.4559e-03],
         [-1.3302e-03,  1.0365e-03,  6.6083e-04]],

        [[-1.3468e-03,  8.5255e-04,  5.1631e-04],
         [-1.3240e-03,  9.8264e-04,  4.8372e-04],
         [-2.0458e-03,  1.1292e-04,  8.6764e-04],
         ...,
         [-1.5469e-03,  1.1453e-03,  8.0484e-04],
         [ 7.3468e-03, -1.7595e-03, -1.5087e-03],
         [-1.3707e-03,  8.3661e-04,  1.0118e-03]],

        ...,

        [[-1

0

### Model evaluation

We compute the performance of our model by first generating the price process, apply the prespecified trading strategies and compare the resulting PnL process using the real and fake data.

In [10]:
from src.evaluation.strategies import log_return_to_price

config_dir = 'src/evaluation/config.yaml'
with open(config_dir) as file:
    eval_config = ml_collections.ConfigDict(yaml.safe_load(file))

generated_sample = log_return_to_price(generated_sample[:eval_size], cv_val_init_price[:eval_size])
cv_val = log_return_to_price(cv_validation_data[:eval_size], cv_val_init_price[:eval_size])

# Check if all generated prices are positive
all_positive = (generated_sample > 0).all()
if not all_positive:
    raise ValueError("Sanity Check Failed: Some fake prices are not positive.")

# Initialize dictionaries to store results
res_dict = {"var_mean": 0., "es_mean": 0., "max_drawback_mean": 0., "cumulative_pnl_mean": 0.}
detailed_res_dict = {}  # Store detailed metrics for each strategy

num_strat = 4

# Perform final evaluation for each strategy
with torch.no_grad():
    for strat_name in ['equal_weight', 'mean_reversion', 'trend_following', 'vol_trading']:
        # Evaluate strategy
        subres_dict = full_evaluation(generated_sample, cv_val, eval_config, strat_name=strat_name)
        
        # Store detailed results for this strategy
        detailed_res_dict[strat_name] = subres_dict
        
        # Accumulate results for mean calculation
        for k in res_dict:
            res_dict[k] += subres_dict[k] / num_strat

# Print detailed results for each strategy
print("Detailed Metrics per Strategy:")
for strat_name, metrics in detailed_res_dict.items():
    print(f"Strategy: {strat_name}")
    for k, v in metrics.items():
        print(f"  {k}: {v}")
        
# Print mean results
print("\nMean Metrics across all Strategies:")
for k, v in res_dict.items():
    print(f"{k}: {v}")

Detailed Metrics per Strategy:
Strategy: equal_weight
  var_mean: 0.027606401592493057
  var_std: 0.0
  es_mean: 0.04181969538331032
  es_std: 0.0
  max_drawback_mean: 0.021213140338659286
  max_drawback_std: 0.0
  cumulative_pnl_mean: 0.025439638644456863
  cumulative_pnl_std: 0.0
Strategy: mean_reversion
  var_mean: 0.09491269290447235
  var_std: 0.0
  es_mean: 0.2067442089319229
  es_std: 0.0
  max_drawback_mean: 0.07910014688968658
  max_drawback_std: 0.0
  cumulative_pnl_mean: 0.07324480265378952
  cumulative_pnl_std: 0.0
Strategy: trend_following
  var_mean: 0.04603852704167366
  var_std: 0.0
  es_mean: 0.03988272324204445
  es_std: 0.0
  max_drawback_mean: 0.033322229981422424
  max_drawback_std: 0.0
  cumulative_pnl_mean: 0.01928303763270378
  cumulative_pnl_std: 0.0
Strategy: vol_trading
  var_mean: 0.07536911219358444
  var_std: 0.0
  es_mean: 0.19228506088256836
  es_std: 0.0
  max_drawback_mean: 0.04561632499098778
  max_drawback_std: 0.0
  cumulative_pnl_mean: 0.0463587865

In [11]:
with open("./submission_bundle/fake_log_return.pkl", "rb") as f:
    loaded_array = pickle.load(f)
train_log_return = torch.tensor(loaded_array)
print(train_log_return.shape)

torch.Size([1600, 24, 3])


C:\Users\Admin\AppData\Local\Temp\ipykernel_35560\2633653836.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_log_return = torch.tensor(loaded_array)


In [12]:
with open("./submission_bundle/model_dict.pkl", "rb") as f:
    loaded_array = pickle.load(f)
print(loaded_array)

OrderedDict([('fc1.weight', tensor([[ 0.3909,  0.4226, -0.1121,  0.4545],
        [-0.1145,  0.1074, -0.2393,  0.2867],
        [ 0.4463, -0.3674,  0.4395,  0.0949],
        ...,
        [-0.1435, -0.4183,  0.1973,  0.3569],
        [ 0.0336, -0.3604, -0.3857, -0.1946],
        [ 0.1518,  0.0176,  0.0961,  0.4735]], device='cuda:0')), ('fc1.bias', tensor([ 2.3402e-01,  1.0555e-01, -9.3098e-02,  2.4039e-01,  1.8428e-01,
        -4.5362e-01, -2.9256e-01,  2.7816e-02,  1.9459e-01,  2.4066e-01,
        -2.9775e-01, -4.9776e-01, -1.4445e-02, -4.7679e-02,  1.4994e-01,
        -1.3702e-01,  3.8459e-01,  4.4216e-02,  3.7017e-01, -3.5095e-01,
         2.0356e-01,  6.8120e-02, -2.4835e-01,  2.5037e-01, -4.8851e-01,
         4.0288e-01,  5.2214e-02,  6.0624e-02,  2.3441e-01,  5.1564e-02,
         2.1891e-01,  3.2120e-01, -3.8697e-01,  3.2881e-01, -6.3166e-02,
        -1.1265e-01,  1.0487e-01,  3.9200e-01, -2.6849e-01, -3.3395e-01,
         3.1463e-01, -2.3712e-01,  1.6613e-01, -6.5126e-02,  1.357